# Pytorch  

#### Работа с Pytorch

## Установка  

https://pytorch.org/ 

```sh
pip3 install torch torchvision torchaudio
```

## Проверка версии  


In [ ]:
# через команду pip
!pip show torch

Name: torch
Version: 2.7.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /home/maksim/miniconda3/envs/condaenv/lib/python3.12/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, setuptools, sympy, triton, typing-extensions
Required-by: torchaudio, torchvision


In [5]:
# Через conda
!conda list | grep torch

torch                       2.7.1            pypi_0           pypi
torchaudio                  2.7.1            pypi_0           pypi
torchvision                 0.22.1           pypi_0           pypi


In [1]:
# Через python код
import torch
print(torch.__version__)


2.7.1+cu126


## Информация о cuda используемой PyTorch

In [6]:
import torch
print(torch.version.cuda)          # версия CUDA, с которой собран PyTorch
print(torch.cuda.is_available())   # доступна ли CUDA на вашем устройстве (True/False)
print(torch.cuda.current_device()) # номер текущего CUDA-устройства
print(torch.cuda.get_device_name(0)) # имя первого CUDA-устройства



12.6
True
0
NVIDIA GeForce RTX 3060


## работа с Pytorch

In [74]:
# создание тензора
import torch
x = torch.tensor([1.0, 2.0, 3.0])
print(x)


tensor([1., 2., 3.])


In [75]:
# для работы с GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    x = x.to(device)
    print(f"Tensor is on device: {x.device}")
else:
    print("CUDA is not available.")


Tensor is on device: cuda:0


### Создание тензоров

In [ ]:
import torch

# Создание тензора из списка
tensor = torch.tensor([[1, 2], [3, 4]])
print(tensor)

# Тензор из нулей
zeros = torch.zeros(3, 4)
print(zeros)

# Тензор из единиц
ones = torch.ones(2, 3)
print(ones)

# Тензор с равномерно распределёнными значениями
linspace = torch.linspace(0, 1, steps=5)
print(linspace)

# Одномерный тензор с диапазоном значений
arange = torch.arange(0, 10, step=2)
print(arange)


### операции

In [76]:
# Арифметические операции поэлементно:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])

print(a + b)  # tensor([5, 7, 9])
print(a * b)  # tensor([4, 10, 18])
print(b / a)  # tensor([4.0000, 2.5000, 2.0000])

# Операции с числами (скалярами):
c = torch.tensor([1, 2, 3])
print(c + 5)  # tensor([6, 7, 8])
print(c * 2)  # tensor([2, 4, 6])

# Индексирование и срезы работают как в numpy:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(x[0, 1])    # 2
print(x[:, 1:3])  # все строки, столбцы с 1 по 2




tensor([5, 7, 9])
tensor([ 4, 10, 18])
tensor([4.0000, 2.5000, 2.0000])
tensor([6, 7, 8])
tensor([2, 4, 6])
tensor(2)
tensor([[2, 3],
        [5, 6]])


### создание тензора, вычисление градиента и перенос на GPU

In [77]:
import torch

# Создаем тензор с включенным отслеживанием градиентов
x = torch.tensor([2.0, 3.0], requires_grad=True)

# Выполняем операцию
y = x ** 2 + 3 * x + 1

# Считаем сумму для удобства
z = y.sum()

# Вычисляем градиенты
z.backward()

print(x.grad)  # tensor([7., 9.]) — производные по x

# Перенос на GPU (если доступен)
if torch.cuda.is_available():
    x_gpu = x.to('cuda')
    print(x_gpu.device)  # cuda:0


tensor([7., 9.])
cuda:0


In [79]:
''' 
y = x ** 2  # y — тензор с несколькими элементами
y.backward()  # вызовет ошибку
'''

y = x ** 2
y.sum().backward()  # теперь y — скаляр, ошибка исчезнет

In [ ]:
y = x ** 2
y.backward(torch.ones_like(y))  # явно передаем градиенты  


In [ ]:
# Если два раза вызвать y.backward(torch.ones_like(y))  будет ошибка
''' 
y.backward(torch.ones_like(y)) 
y.backward(torch.ones_like(y)) 
'''
# По умолчанию, после вызова .backward(), вычислительный граф, который PyTorch строит для отслеживания операций и вычисления градиентов, удаляется.

In [82]:
import torch

x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * 2

# Первый вызов: сохраняем граф
y.backward(torch.ones_like(y), retain_graph=True)
print("Градиенты после первого backward:", x.grad)

# Сбрасываем градиенты (важно, чтобы не накапливались)
x.grad.zero_()

# Второй вызов: используем сохраненный граф
y.backward(torch.ones_like(y)) # retain_graph=True здесь не обязательно, если это последний вызов
print("Градиенты после второго backward:", x.grad)


Градиенты после первого backward: tensor([2., 2., 2.])
Градиенты после второго backward: tensor([2., 2., 2.])
